In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import geopandas as gpd
from nycschools import geo, datasets, dataloader as dl

In [8]:
city = geo.load_city_footprints()
# feather = "/home/mxc/Projects/school-data-portal/school-data/nyc-footprints.feather"
# df = gpd.read_feather(feather)
# df

loading file /home/mxc/Projects/school-data-portal/school-data/nyc-footprints.feather
geo file True
reading feather as gdf


In [5]:
file = "../_data/Building Footprints_20250409.geojson" # downloaded on 2025-04-09
df = gpd.read_file(file)
df.head()
df.to_feather("../_data/dob.feather")
df.to_parquet("../_data/dob.parquet")


In [3]:
dob = dl.load("/home/mxc/Projects/school-data-portal/drafts/_data/dob.feather", gdf=True)
dob.head()

,name,base_bbl,shape_area,heightroof,mpluto_bbl,cnstrct_yr,globalid,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,shape_len,geometry
0,None,3065220021,0.0,29.74985318,3065220021,1925,{31298F86-3088-4F53-B3DB-71A9EFA6FA1F},Constructed,2100,40,Photogramm,3170958,2017-08-22,96807,0.0,"MULTIPOLYGON (((-73.96665 40.626, -73.96685 40..."
1,None,5012640036,0.0,22.63,5012640036,1965,{F5F8CDA5-69E2-46F8-8F69-BA95C025B520},Constructed,2100,39,Photogramm,5028452,2017-08-22,326368,0.0,"MULTIPOLYGON (((-74.1679 40.63936, -74.1679 40..."
2,None,5060190091,0.0,35.76,5060190091,1970,{9F644794-F72C-4582-9E5E-B337E2B97068},Constructed,2100,51,Photogramm,5078368,2017-08-22,746627,0.0,"MULTIPOLYGON (((-74.19511 40.55611, -74.1952 4..."
3,None,3086910048,0.0,37.5,3086910048,1928,{F916B22D-E25B-44AE-9FA9-2A51191B9CDF},Constructed,2100,6,Photogramm,3245111,2017-08-22,786626,0.0,"MULTIPOLYGON (((-73.96113 40.57744, -73.96115 ..."
4,None,4075020005,0.0,18.01511294,4075020005,1950,{525F2C24-616B-4F29-98A3-8FEA5D4B1A7D},Constructed,2100,93,Photogramm,4161096,2017-08-22,746409,0.0,"MULTIPOLYGON (((-73.75422 40.75591, -73.75417 ..."


In [20]:
loc = pd.read_csv( "https://data.cityofnewyork.us/resource/wg9x-4ke6.csv?$limit=1000000")
loc.fiscal_year.max()

np.int64(2020)

In [42]:
foot = dob.copy()
foot["bbl"] = foot.base_bbl.astype(str)
foot["mpluto_bbl"] = foot.mpluto_bbl.astype(str)
geojsonurl = datasets.urls["school_geo"]["url"]
school_loc = geo.get_locations()

school_foot = foot.merge(school_loc[["dbn","bbl"]], on="bbl", how="inner")
crs = school_foot.crs

multi = school_foot[school_foot["dbn"].duplicated(keep=False)].copy()
multi = multi.dissolve(by="dbn").reset_index()

school_foot = school_foot[~school_foot.dbn.isin(multi.dbn)]
school_foot = pd.concat([school_foot, multi], ignore_index=True)
school_foot = gpd.GeoDataFrame(school_foot, crs=crs)



In [ ]:
from nycschools import schools
df = schools.load_school_demographics()
df = df[df.ay == df.ay.max()]


(1892, 2075)

In [49]:
f = school_foot.merge(df[["dbn", "school_name"]], on="dbn", how="inner")
t = df[~df.dbn.isin(f.dbn)][["dbn", "school_name", "district", "school_type", "school_level"]]
t.school_type.value_counts()

school_type
community    53
charter      50
transfer      8
d75           2
Name: count, dtype: int64

In [55]:
school_foot.explore(tooltip="cnstrct_yr")
school_foot.columns

Index(['name', 'base_bbl', 'shape_area', 'heightroof', 'mpluto_bbl',
       'cnstrct_yr', 'globalid', 'lststatype', 'feat_code', 'groundelev',
       'geomsource', 'bin', 'lstmoddate', 'doitt_id', 'shape_len', 'geometry',
       'bbl', 'dbn'],
      dtype='object')

In [63]:
# old = geo.load_school_footprints()
# old.dbn.nunique(), school_foot.dbn.nunique()
f = datasets.urls["building_footprints"]["school_footprints_file"]
out = f"{dl.get_data_dir()}/{f}"
school_foot.to_file(out, driver="GeoJSON")